<a href="https://colab.research.google.com/github/frank-morales2020/Cloud_curious/blob/master/TTPA_EXA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install crewai -q
!pip install exa-py -q
!pip install google-colab -q
!pip install openai -q
!pip install crewai[tools] -q

In [21]:
from IPython import get_ipython
from IPython.display import display
import os
from crewai import Agent, Task, Crew, Process
from crewai.tools import BaseTool
from exa_py import Exa

import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
os.environ["EXA_API_KEY"] = userdata.get('EXA_API_KEY')

# --- 0. Set Up Your Environment Variables (IMPORTANT!) ---
# Before running this script, you MUST set the following environment variables.
# You can do this in your terminal:
#
# export EXA_API_KEY="your_actual_exa_api_key_here"
# export OPENAI_API_KEY="your_actual_openai_api_key_here"
#
# If you are using a local LLM (e.g., with Ollama):
# export OPENAI_API_BASE='http://localhost:11434/v1'
# export OPENAI_MODEL_NAME='openhermes' # Or the name of your specific local model

# --- 1. Define Custom Exa Tools ---
# These classes wrap Exa's functionalities into CrewAI-compatible tools.

class ExaSearchTool(BaseTool):
    """
    A custom tool for performing semantic searches using the Exa API.
    Returns titles, URLs, and IDs of relevant web pages.
    """
    name: str = "Exa Search Tool"
    description: str = "Searches the web using Exa for relevant information, returning titles, URLs, and IDs."
    exa_instance: Exa = None

    def __init__(self):
        super().__init__()
        try:
            self.exa_instance = Exa(api_key=os.environ["EXA_API_KEY"])
        except KeyError:
            raise ValueError(
                "EXA_API_KEY environment variable not set. Please set it "
                "to your Exa API key."
            )
        except Exception as e:
            raise Exception(f"Failed to initialize ExaSearchTool: {e}")

    def _run(self, query: str, limit: int = 5, include_domains: list = None) -> list:
        """
        Executes the Exa search.

        Args:
            query (str): The search query (e.g., "flights from New York to London").
            limit (int): Maximum number of search results to return.
            include_domains (list): Optional list of domains to prioritize results from.

        Returns:
            list: A list of dictionaries, each containing 'title', 'url', and 'id'
                  of the search results. Returns an empty list if no results or error.
        """
        if not self.exa_instance:
            return "Exa API not initialized due to missing API key."

        try:
            print(f"\n[ExaSearchTool] Searching Exa for query: '{query}'...")
            response = self.exa_instance.search(
                query=query,
                num_results=limit,
                include_domains=include_domains if include_domains else [],
            )

            results = []
            if response and response.results:
                for result in response.results:
                    results.append({
                        "title": result.title,
                        "url": result.url,
                        "id": result.id
                    })
                print(f"[ExaSearchTool] Found {len(results)} results for '{query}'.")
            else:
                print(f"[ExaSearchTool] No relevant results found for '{query}'.")
            return results
        except Exception as e:
            return f"An error occurred during Exa search: {e}"

class ExaContentRetrieveTool(BaseTool):
    """
    A custom tool for retrieving the full text content of a web page
    by its Exa result ID.
    """
    name: str = "Exa Content Retrieval Tool"
    # Corrected the assignment of the description - Added str type hint
    description: str = "Retrieves the full text content of a web page by its Exa result ID."
    exa_instance: Exa = None

    def __init__(self):
        super().__init__()
        try:
            self.exa_instance = Exa(api_key=os.environ["EXA_API_KEY"])
        except KeyError:
            raise ValueError(
                "EXA_API_KEY environment variable not set. Please set it "
                "to your Exa API key."
            )
        except Exception as e:
            raise Exception(f"Failed to initialize ExaContentRetrieveTool: {e}")

    def _run(self, result_id: str) -> str:
        """
        Retrieves the full text content of a search result by its ID.

        Args:
            result_id (str): The ID of the search result from Exa.

        Returns:
            str: The full text content of the page, or None if not found/error.
        """
        if not self.exa_instance:
            return "Exa API not initialized due to missing API key."
        try:
            print(f"[ExaContentRetrieveTool] Getting content for ID: {result_id}...")
            response = self.exa_instance.get_contents(ids=[result_id])
            if response and response.results:
                content = response.results[0].text
                print(f"[ExaContentRetrieveTool] Content retrieved for ID: {result_id}.")
                return content
            else:
                print(f"[ExaContentRetrieveTool] No content found for ID: {result_id}.")
                return "No content found for this ID."
        except Exception as e:
            return f"An error occurred while getting content: {e}"

# Instantiate our custom Exa tools. This will raise an error if EXA_API_KEY is not set.
try:
    exa_search_tool = ExaSearchTool()
    exa_content_tool = ExaContentRetrieveTool()
except ValueError as e:
    print(f"Tool initialization error: {e}")
    print("Please ensure your EXA_API_KEY environment variable is set and restart the script.")
    exit() # Exit if crucial API key is missing

# --- 2. Define the Agents ---

# Flight & Transportation Agent (FTA)
flight_agent = Agent(
    role='Flight Information Specialist',
    goal='Accurately identify relevant flight news, policies, and reliable booking links for given routes and dates.',
    backstory=(
        "You are an expert in air travel, meticulously researching current airline policies, "
        "flight search best practices, and airport information. Your precision ensures "
        "travelers receive the most accurate and helpful information for their flight planning."
    ),
    tools=[exa_search_tool, exa_content_tool], # Equip with our custom Exa tools
    verbose=False, # Set to False to reduce output and potentially avoid recursion error
    allow_delegation=False # This agent performs the search directly
)

# Travel Research Orchestrator Agent
# This agent acts as a high-level planner and can delegate to other specialized agents.
orchestrator_agent = Agent(
    role='Travel Research Orchestrator',
    goal='Coordinate information gathering for comprehensive travel planning, delegating to specialized agents and synthesizing findings.',
    backstory=(
        "You are the central brain of a travel planning AI, capable of understanding complex user requests, "
        "breaking them down, and intelligently delegating tasks to expert agents. You then synthesize "
        "their findings into a coherent, actionable travel brief."
    ),
    tools=[], # This agent might not use Exa directly but delegates tasks that do.
              # If it needs to do initial broad searches, it could also have exa_search_tool.
    verbose=False, # Set to False to reduce output and potentially avoid recursion error
    allow_delegation=True # Crucial for delegation in a hierarchical process
)

# --- 3. Define the Tasks ---

# Task 1: For the Flight Information Specialist
# This task requires the agent to use its Exa tools.
flight_research_task = Task(
    description=(
        "Search for general flight information for a trip from **Montreal to Paris** "
        "during **July 2025**. Focus on finding reputable flight aggregators, "
        "news about typical flight prices, and any common airlines on this route. "
        "If a specific article seems highly relevant for price trends, retrieve its content."
    ),
    expected_output=(
        "A bullet-point summary of general flight information for Montreal to Paris in July 2025, "
        "including typical price ranges (if found), common airlines, and direct vs. connecting options. "
        "Include 2-3 relevant URLs from reputable flight search sites or travel news."
    ),
    agent=flight_agent,
    tools=[exa_search_tool, exa_content_tool] # Ensure task explicitly uses the tools
)

# Task 2: For the Orchestrator Agent
# This task primarily involves delegating and summarizing.
orchestrator_summary_task = Task(
    description=(
        "Review the flight research findings provided by the 'Flight Information Specialist'. "
        "Then, compile a concise, actionable summary for the user about potential flight options "
        "from Montreal to Paris in July 2025 based on the research. "
        "Suggest next steps for the user to book their flights."
    ),
    expected_output=(
        "A user-friendly summary about flight possibilities from Montreal to Paris in July 2025. "
        "This summary should include: typical prices, recommended airlines/routes, and "
        "direct links or advice on where to book the flights. End with a polite call to action."
    ),
    agent=orchestrator_agent,
    # No direct Exa tools needed here, as it processes results from other agents
)

# --- 4. Create and Run the Crew ---

# Instantiate your crew with the agents and tasks
# Using a hierarchical process allows the orchestrator to delegate tasks.
travel_planning_crew = Crew(
    # Remove the manager_agent from the agents list
    agents=[flight_agent],
    tasks=[flight_research_task, orchestrator_summary_task], # All tasks in the workflow
    process=Process.hierarchical, # Orchestrator can delegate tasks
    manager_agent=orchestrator_agent, # Specify the orchestrator agent as the manager
    verbose=False # Set to False to reduce output and potentially avoid recursion error
)

print("\n\n############################################")
print("## Starting the Gemini 2.0 TTPA Crew... ##")
print("############################################\n")

# Kickoff the crew to start the execution
final_result = travel_planning_crew.kickoff()

print("\n############################################")
print("## TTPA Crew Finished! Final Output: ##")
print("############################################\n")
print(final_result)



############################################
## Starting the Gemini 2.0 TTPA Crew... ##
############################################


[ExaSearchTool] Searching Exa for query: 'Montreal to Paris flights July 2025 flight information'...

[ExaSearchTool] Searching Exa for query: 'Montreal to Paris flights July 2025 typical prices airlines'...

[ExaSearchTool] Searching Exa for query: 'Montreal to Paris flights July 2025 airlines prices direct connecting options'...

[ExaSearchTool] Searching Exa for query: 'Montreal to Paris flights July 2025 airlines prices'...

[ExaSearchTool] Searching Exa for query: 'Montreal to Paris flight options July 2025'...

[ExaSearchTool] Searching Exa for query: 'Montreal to Paris July 2025 flight information price aggregator airlines'...

[ExaSearchTool] Searching Exa for query: 'Montreal Paris flight July 2025 direct connecting options prices'...

[ExaSearchTool] Searching Exa for query: 'Montreal to Paris flights July 2025 aggregation airline informatio